In [31]:
import tensorflow as tf 
import numpy as np 
import matplotlib.pyplot as plt
from keras.models import Model 
from keras.layers import Input, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score 

In [32]:
def system_dynamics(x_t, u_t):
    """
    System dynamics: x_t+1 = x_t + 0.5*u_t
    """
    return x_t + 0.5 * u_t

def reward(x, u):
    return -(x**2 + 0.001 * u**2)

n_steps = 20
gamma = 1.0
history = []
np.random.seed(42)

In [33]:
# Define the actor model
inputs = Input(shape=(1,))
# dense = Dense(16, activation='relu')(inputs)
outputs = Dense(1, activation='linear')(inputs)
actor_model = Model(inputs, outputs)

# Define the critic model
inputs = Input(shape=(2,))
dense = Dense(16, activation='relu')(inputs)
outputs = Dense(1, activation='linear')(dense)
critic_model = Model(inputs, outputs)

# Optimizers
actor_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
critic_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

In [34]:
for epoch in range(4000):
    x_t = np.random.uniform(-1, 1)
    total_reward = 0
    episode_memory = []
    for _ in range(n_steps):
        x_tensor = tf.convert_to_tensor([[x_t]], dtype=tf.float32)
        u_t = actor_model(x_tensor).numpy()[0, 0] + np.random.normal(0, 1)
        x_next = system_dynamics(x_t, u_t)
        r_t = reward(x_t, u_t)
        episode_memory.append((x_t, u_t, r_t, x_next))

    states, actions, rewards, next_states = zip(*episode_memory)
    x_t = tf.convert_to_tensor(np.array(states).reshape(-1, 1), dtype=tf.float32)
    u_t = tf.convert_to_tensor(np.array(actions).reshape(-1, 1), dtype=tf.float32)
    r_t = tf.convert_to_tensor(np.array(rewards).reshape(-1, 1), dtype=tf.float32)
    x_next = tf.convert_to_tensor(np.array(next_states).reshape(-1,1 ), dtype=tf.float32)

    with tf.GradientTape() as tape_critic:  
        q_pred = critic_model(tf.concat([x_t, u_t], axis=1))
        u_next = actor_model(x_next)
        q_next = critic_model(tf.concat([x_next, u_next], axis=1))
        q_target = r_t + gamma * q_next
        critic_loss = tf.reduce_mean(q_pred - q_target)**2
    with tf.GradientTape() as tape_actor:
        u_pred = actor_model(x_t)
        q_val = critic_model(tf.concat([x_t, u_pred], axis=1))
        actor_loss = -tf.reduce_mean(q_val)
    
    critic_grads = tape_critic.gradient(critic_loss, critic_model.trainable_variables)
    actor_grads = tape_actor.gradient(actor_loss, actor_model.trainable_variables)
    
    critic_optimizer.apply_gradients(zip(critic_grads, critic_model.trainable_variables))
    actor_optimizer.apply_gradients(zip(actor_grads, actor_model.trainable_variables))
    x_t = x_next
    total_reward += r_t.numpy()[0, 0]
    history.append(total_reward)

    if (epoch + 1) % 50 == 0:
        print(f"Epoch {epoch+1}, Critic Loss: {critic_loss.numpy():.4f}, Actor Loss: {actor_loss.numpy():.4f}, Total Reward: {total_reward:.2f}")
     


Epoch 50, Critic Loss: 0.0030, Actor Loss: 0.1080, Total Reward: -0.19
Epoch 100, Critic Loss: 0.0013, Actor Loss: 0.1169, Total Reward: -0.17
Epoch 150, Critic Loss: 0.0037, Actor Loss: 0.2885, Total Reward: -0.28
Epoch 200, Critic Loss: 0.0088, Actor Loss: -0.0016, Total Reward: -0.00
Epoch 250, Critic Loss: 0.0040, Actor Loss: 0.3220, Total Reward: -0.38
Epoch 300, Critic Loss: 0.0943, Actor Loss: 0.3755, Total Reward: -0.71
Epoch 350, Critic Loss: 0.0063, Actor Loss: 0.3002, Total Reward: -0.23
Epoch 400, Critic Loss: 0.0002, Actor Loss: 0.3701, Total Reward: -0.29
Epoch 450, Critic Loss: 0.0078, Actor Loss: 0.1586, Total Reward: -0.17
Epoch 500, Critic Loss: 0.2423, Actor Loss: 0.7791, Total Reward: -0.93
Epoch 550, Critic Loss: 0.0858, Actor Loss: 0.7713, Total Reward: -0.78
Epoch 600, Critic Loss: 0.1286, Actor Loss: 0.4586, Total Reward: -0.55
Epoch 650, Critic Loss: 0.0105, Actor Loss: 0.0940, Total Reward: -0.06
Epoch 700, Critic Loss: 0.0191, Actor Loss: 0.1039, Total Reward

0.9837705492973328

In [35]:
x_test = np.linspace(-1, 1, 100).reshape(-1, 1).astype(np.float32)

# Predict using the trained actor
u_pred = actor_model.predict(x_test, verbose=0)

# Define correct analytical policy
u_analytical = -2 * x_test

r2 = r2_score(u_analytical, u_pred)

# Plot
plt.figure(figsize=(9, 6))
plt.plot(x_test, u_pred, label='Learned Policy (Actor NN)')
plt.plot(x_test, u_analytical, '--', label='Analytical Policy: u = -2x', color='orange')
plt.xlabel('x', fontsize=17)
plt.ylabel('u', fontsize=17)
plt.title(f'Control Policy Discovered via RL R² = {r2:.4f}', fontsize=17)
plt.grid()
plt.legend(fontsize=17)
plt.show()
r2